Building the SmartTag models
===


# Initial checks

In [ ]:
source .venv/bin/activate

In [ ]:
neo4j start

In [ ]:
neo4j status

In [ ]:
ls models

In [ ]:
ls data

In [ ]:
ls data4th

In [ ]:
smtag-neo2xml --help

In [12]:
smtag-convert2th --help

usage: smtag-convert2th [-h] [-c PATH] [-f FILENAMEBASE] [-T] [-b]
                        [-X ITERATIONS] [-v] [-L LENGTH] [-W] [-S] [-d]
                        [-p PADDING] [-w WORKING_DIRECTORY]

Reads xml and transform into tensor format.

optional arguments:
  -h, --help            show this help message and exit
  -c PATH, --path PATH  path to the source compendium of xml files (default:
                        demo_xml)
  -f FILENAMEBASE, --filenamebase FILENAMEBASE
                        namebase to save converted trainset and features set
                        files (default: )
  -T, --testset         use the testset instead of the trainset (default:
                        False)
  -b, --brat            Use brat files instead of xml files (default: False)
  -X ITERATIONS, --iterations ITERATIONS
                        number of times each example is sampled (default: 5)
  -v, --verbose         verbosity (default: False)
  -L LENGTH, --length LENGTH
                      

In [ ]:
smtag-meta --help

In [ ]:
smtag-eval --help

In [ ]:
smtag-predict --help

In [ ]:
tensorboard --logdir runs &

# A. Data generation

## Small molecules

Enrich examples for small molecules (`-y`). Generate the data with `-5X` sampling of each example of length `-L1200` characters.

In [ ]:
smtag-neo2xml -l1000 -y molecule -f small_molecule > /dev/null &2>1

In [ ]:
smtag-convert2th -X5 -L1200 -X5 -f 5X_L1200_small_molecule > /dev/null &2>1

Prepare test set (`-T`) for later:

In [ ]:
smtag-convert2th -X5 -L1200 -X5 -f 5X_L1200_small_molecule -T > /dev/null &2>1

## Gene and proteins

In [ ]:
smtag-neo2xml -l1000 -y gene,protein -f gene_protein

smtag-convert2th -X5 -L1200 -X5 -c gene_protein -f 5X_L1200_gene_protein

In [ ]:
 smtag-meta -Z128 -E150 -R0.001 -f 5X_L1200_gene_protein_train

## Diseases

Using a corpus in the brat format (`-b`) instead of the xml format.

In [ ]:
smtag-convert2th -L1200 -X5 -c ncbi_disease

## Intervention and measurements on geneproducts given reporter

Anonymize gene and proteins (-`A`) but do not anonymize reporters (`-AA`). Keep roles only for genes and proteins (`-s`).

In [ ]:
smtag-neo2xml -l1000 -y gene,protein -A gene,protein -AA reporter -s

# Intervention and measurements on small molecules

# Report on geneproducts

Select enrich for protein and genes (`-y`) and reporters (`-r`). Remove all tags except genes and proteins (`-e`).

In [ ]:
smtag-neo2xml -l1000 -y gene,protein -r reporter -e

# B. Moving training data to Amazon EFS

In [ ]:
    cd ../cloud

In [ ]:
    scp -i basicLinuxAMI.pem 5X_L1200_small_molecule_train.zip ec2-user@smtag-web:/efs/smtag/data/

In [ ]:
    scp -i basicLinuxAMI.pem 5X_L1200_gene_protein_train.zip ec2-user@smtag-web:/efs/smtag/data/

In [ ]:
    scp -i basicLinuxAMI.pem 5X_L1200_subcell_train.zip ec2-user@smtag-web:/efs/smtag/data/

In [ ]:
    scp -i basicLinuxAMI.pem 5X_L1200_cell_train.zip ec2-user@smtag-web:/efs/smtag/data/

In [ ]:
    scp -i basicLinuxAMI.pem 5X_L1200_tissue_train.zip ec2-user@smtag-web:/efs/smtag/data/

In [ ]:
    scp -i basicLinuxAMI.pem 5X_L1200_organism_train.zip ec2-user@smtag-web:/efs/smtag/data/

In [ ]:
    scp -i basicLinuxAMI.pem 5X_L1200_exp_assay_train.zip ec2-user@smtag-web:/efs/smtag/data/

In [ ]:
    scp -i basicLinuxAMI.pem 5X_L1200_disease_train.zip ec2-user@smtag-web:/efs/smtag/data/

# C. Training models

# Entity models

### Small molecule model

200 epoch with learning rate 0.001 and batch size 128

In [ ]:
smtag-meta -Z128 -E200 -R0.001 -f 5X_L1200_small_molecule_train

# D. Benchmarking on the test set

Without tokenization (option `-T`)

In [ ]:
smtag-eval -f 5X_L1200_small_molecule_w_test -m 5X_L1200_small_molecule -T